In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras import regularizers
import nltk
import re
import csv
import pandas as pd
import numpy as np
import pickle
import os
import io
import itertools
import re
from collections import Counter
import requests
from bs4 import BeautifulSoup
import string
from ekphrasis.classes import segmenter, spellcorrect
from sklearn import linear_model
from sklearn.metrics import accuracy_score
import binascii
import fasttext

from fast_text import fast_text
from glove_helper import load_glove_model, average_embeddings
from preprocessing_methods import *

Using TensorFlow backend.
/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Reading twitter - 1grams ...
Reading twitter - 2grams ...
Reading english - 1grams ...


# `Load Data`

In [2]:
def load_file(filename):
    data = []
    file = open(filename, "r")
    for line in file:
        data.append(line)
    return data

In [3]:
pos_tweets = pd.DataFrame(load_file('twitter-dataset/train_pos_full.txt'), columns=['tweet'])
pos_tweets['sentiment'] = 1

neg_tweets = pd.DataFrame(load_file('twitter-dataset/train_neg_full.txt'), columns=['tweet'])
neg_tweets['sentiment'] = 0

train_tweets = pd.concat([pos_tweets, neg_tweets], axis=0)
test_tweets = train_tweets.sample(frac=0.01)
train_tweets = train_tweets.sample(frac=0.1)

In [4]:
print("There are %d train and %d test Tweet samples."%(len(train_tweets),len(test_tweets)))

There are 250000 train and 25000 test Tweet samples.


# `Pre-process`

In [5]:
def grid_methods(train_tweets, test_tweets, method_list):  
    
    train_tweets_copy = train_tweets.copy()
    test_tweets_copy = test_tweets.copy()
    
    print("Pipeline < " + " > ".join([str(method).split()[1] for method in method_list]) + "\n")
    
    for method in method_list:
        print("Applying < %s"%str(method).split()[1])
        train_tweets_copy['tweet'] = train_tweets_copy['tweet'].apply(method)
        test_tweets_copy['tweet'] = test_tweets_copy['tweet'].apply(method)
    
    train_tweets_copy['tweet'].replace('', np.nan, inplace=True)
    test_tweets_copy['tweet'].replace('', np.nan, inplace=True)
    train_tweets_copy = train_tweets_copy.dropna()
    test_tweets_copy = test_tweets_copy.dropna()
    
    print("\nDone!")
    return(train_tweets_copy, test_tweets_copy)

In [6]:
selected_methods = [replace_contraction, replace_emoji, replace_slang, emphasize_sentiment_words, filter_digits]
train_tweets_cleaned, test_tweets_cleaned  = grid_methods(train_tweets, test_tweets, selected_methods)

Pipeline < replace_contraction > replace_emoji > replace_slang > emphasize_sentiment_words > filter_digits

Applying < replace_contraction
Applying < replace_emoji
Applying < replace_slang
Applying < emphasize_sentiment_words
Applying < filter_digits

Done!


# `Text to Vector Methods`

## `Fasttext`

`Internal Prediction Model`

In [7]:
pred = fast_text(train_tweets_cleaned, test_tweets_cleaned)
pred = np.array(pred)
acc_list = [1 if el else 0 for el in list(test_tweets_cleaned['sentiment']) == pred]
acc = sum(acc_list) / len(acc_list)
print("Accuracy > %f"%acc)

Accuracy > 0.824720


`Create Word Embeddings`

In [ ]:
fasttext_model = fasttext.load_model('embeddings/fasttext_model.bin')

In [ ]:
train_sentences = list(train_tweets_cleaned['tweet'])
test_sentences = list(test_tweets_cleaned['tweet'])

vector_length = 200
train_features = average_embeddings(train_sentences, fasttext_model, vector_length=vector_length)
test_features = average_embeddings(test_sentences, fasttext_model, vector_length=vector_length)

## `Glove`

`Create Word Embeddings`

In [8]:
glove_model = load_glove_model("embeddings/glove/glove.twitter.27B.200d.txt")

In [9]:
train_sentences = list(train_tweets_cleaned['tweet'])
test_sentences = list(test_tweets_cleaned['tweet'])

vector_length = 200
train_features = average_embeddings(train_sentences, glove_model, vector_length=vector_length)
test_features = average_embeddings(test_sentences, glove_model, vector_length=vector_length)

## `Bag Of Words`

`Create Tweet Vectors`

In [ ]:
def sentence_to_vector(text, vector_length=50):
    return_vector = [0] * vector_length
    for word in text.split():
        index = int(binascii.hexlify(word.encode("utf-8")), 16) % vector_length
        return_vector[index] += 1
    return return_vector

In [ ]:
vector_length = 50
train_features = list(train_tweets_cleaned['tweet'].apply(sentence_to_vector))
test_features = list(test_tweets_cleaned['tweet'].apply(sentence_to_vector))

# `Prediction Model(s)`

In [10]:
train_labels = np.array(train_tweets_cleaned['sentiment'])
test_labels = np.array(test_tweets_cleaned['sentiment'])

train_labels = [[0, 1] if label == 1 else [1, 0] for label in train_labels]
test_labels = [[0, 1] if label == 1 else [1, 0] for label in test_labels]

## `Logistic Regression - Scikit-Learn`

In [ ]:
lr = linear_model.LogisticRegression(C=1e5)
lr.fit(train_features, np.array(train_tweets_cleaned['sentiment']))

In [ ]:
accuracy_score(np.array(test_tweets_cleaned['sentiment']), lr.predict(test_features))

## `Neural Network - Keras`

In [11]:
model = Sequential()
model.add(Dense(units=64, activation='relu', input_dim=vector_length))
model.add(Dense(units=2, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(lr=0.05, momentum=0.9, nesterov=True))

In [12]:
model.fit(np.array(train_features), np.array(train_labels), epochs=75, batch_size=128)

Epoch 1/75
250000/250000 [==============================] - 13s 51us/step - loss: 0.4529
Epoch 2/75
250000/250000 [==============================] - 10s 39us/step - loss: 0.4221
Epoch 3/75
250000/250000 [==============================] - 7s 30us/step - loss: 0.4141
Epoch 4/75
250000/250000 [==============================] - 7s 26us/step - loss: 0.4081
Epoch 5/75
250000/250000 [==============================] - 7s 26us/step - loss: 0.4041
Epoch 6/75
250000/250000 [==============================] - 7s 27us/step - loss: 0.4006
Epoch 7/75
250000/250000 [==============================] - 9s 35us/step - loss: 0.3982
Epoch 8/75
250000/250000 [==============================] - 7s 28us/step - loss: 0.3963
Epoch 9/75
250000/250000 [==============================] - 7s 27us/step - loss: 0.3938
Epoch 10/75
250000/250000 [==============================] - 7s 27us/step - loss: 0.3923
Epoch 11/75
250000/250000 [==============================] - 7s 29us/step - loss: 0.3905
Epoch 12/75
250000/250000 [=

In [13]:
pred = np.argmax(model.predict(np.array(test_features), batch_size=128), axis=1)
acc_list = [1 if b else 0 for b in np.array(test_tweets_cleaned['sentiment']) == pred]
acc = sum(acc_list) / len(acc_list)
print("Accuracy > %f"%acc)

Accuracy > 0.820200


# `Submission`

In [14]:
kaggle_test_tweets = pd.DataFrame(load_file('twitter-dataset/test_data.txt'), columns=['tweet'])
kaggle_test_tweets['tweet'] = kaggle_test_tweets['tweet'].apply(lambda tweet: tweet.split(',', 1)[-1])
kaggle_test_sentences = list(kaggle_test_tweets['tweet'])
kaggle_test_features = average_embeddings(kaggle_test_sentences, glove_model)

In [ ]:
# Fasttext Internal
pred = fast_text(train_tweets_cleaned, kaggle_test_tweets)
pred = np.array(pred)

In [15]:
# Word Embeddings
pred = np.argmax(model.predict(np.array(kaggle_test_features), batch_size=128), axis=1)

In [16]:
def create_csv_file(results,filepath):
    with open(filepath, 'w') as file:
        fieldnames = ['Id', 'Prediction']
        writeFile = csv.DictWriter(file, delimiter=",", fieldnames=fieldnames)
        writeFile.writeheader()
        id_ = 1
        for result in results:
            writeFile.writerow({'Id':int(id_),'Prediction':result})
            id_ += 1

In [17]:
create_csv_file(pred,'submission.csv')